# Alpha 031 之方法論

此檔案紀錄我對於Alpha031的一些學習過程以及實際操作上的想法、問題，以及驗證後的結果。

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline


import numpy as np
import pandas as pd

import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('whitegrid')

## Load Data

對於資料的操作部分：

我使用yf.download 去下載了S&P500 從'2000-01-01', '2022-08-31'的歷史資料
並且存成'S&P500.h5' 之hdf資料格式

以下先把資料讀進來

In [4]:
data = pd.read_hdf('C:\\Users\\2022080101\\data\\S&P500.h5', 'S&P500')

In [5]:
data

Adj Close       Close        High         Low        Open  \
    Date                                                                     
A   1999-12-31   47.400246   55.302216   57.179901   54.542202   56.866955   
    2000-01-03   44.143158   51.502148   56.464592   48.193848   56.330471   
    2000-01-04   40.771103   47.567955   49.266811   46.316166   48.730328   
    2000-01-05   38.242058   44.617310   47.567955   43.141991   47.389126   
    2000-01-06   36.785965   42.918453   44.349072   41.577251   44.080830   
...                    ...         ...         ...         ...         ...   
ZTS 2022-08-24  162.630005  162.630005  163.869995  162.089996  163.199997   
    2022-08-25  165.529999  165.529999  165.639999  162.360001  163.550003   
    2022-08-26  159.889999  159.889999  166.240005  159.660004  165.710007   
    2022-08-29  157.869995  157.869995  159.139999  157.529999  158.309998   
    2022-08-30  157.080002  157.080002  158.750000  156.759995  158.479996   

                   Volume  
    Date                   
A   1999-12-31  1931197.0  
    2000-01-03  4674353.0  
    2000-01-04  4765083.0  
    2000-01-05  5758642.0  
    2000-01-06  2534434.0  
...                   ...  
ZTS 2022-08-24  1206400.0  
    2022-08-25  1088800.0  
    2022-08-26  1468000.0  
    2022-08-29  1236900.0  
    2022-08-30  1918800.0  

[2545560 rows x 6 columns]

In [6]:
data.index.names = ['Ticker', 'Date']

In [7]:
data

Adj Close       Close        High         Low        Open  \
Ticker Date                                                                     
A      1999-12-31   47.400246   55.302216   57.179901   54.542202   56.866955   
       2000-01-03   44.143158   51.502148   56.464592   48.193848   56.330471   
       2000-01-04   40.771103   47.567955   49.266811   46.316166   48.730328   
       2000-01-05   38.242058   44.617310   47.567955   43.141991   47.389126   
       2000-01-06   36.785965   42.918453   44.349072   41.577251   44.080830   
...                       ...         ...         ...         ...         ...   
ZTS    2022-08-24  162.630005  162.630005  163.869995  162.089996  163.199997   
       2022-08-25  165.529999  165.529999  165.639999  162.360001  163.550003   
       2022-08-26  159.889999  159.889999  166.240005  159.660004  165.710007   
       2022-08-29  157.869995  157.869995  159.139999  157.529999  158.309998   
       2022-08-30  157.080002  157.080002  158.750000  156.759995  158.479996   

                      Volume  
Ticker Date                   
A      1999-12-31  1931197.0  
       2000-01-03  4674353.0  
       2000-01-04  4765083.0  
       2000-01-05  5758642.0  
       2000-01-06  2534434.0  
...                      ...  
ZTS    2022-08-24  1206400.0  
       2022-08-25  1088800.0  
       2022-08-26  1468000.0  
       2022-08-29  1236900.0  
       2022-08-30  1918800.0  

[2545560 rows x 6 columns]

# Alpha 31

In [8]:
data.unstack(level = 0)

Adj Close                                                         \
Ticker               A    AAL         AAP        AAPL        ABBV         ABC   
Date                                                                            
1999-12-31   47.400246    NaN         NaN    0.783791         NaN    2.934864   
2000-01-03   44.143158    NaN         NaN    0.853355         NaN    3.007330   
2000-01-04   40.771103    NaN         NaN    0.781409         NaN    2.802011   
2000-01-05   38.242058    NaN         NaN    0.792843         NaN    3.019407   
2000-01-06   36.785965    NaN         NaN    0.724232         NaN    3.248882   
...                ...    ...         ...         ...         ...         ...   
2022-08-24  133.669998  13.99  178.322937  167.529999  137.910004  148.229996   
2022-08-25  136.009995  14.42  179.393417  170.029999  139.330002  150.419998   
2022-08-26  129.869995  13.74  172.058685  163.619995  136.350006  146.190002   
2022-08-29  128.110001  13.51  169.035583  161.380005  135.710007  147.490005   
2022-08-30  128.279999  13.33  168.966217  158.910004  135.550003  145.429993   

                                                            ...     Volume  \
Ticker            ABMD         ABT         ACN        ADBE  ...       WYNN   
Date                                                        ...              
1999-12-31   18.375000    9.655472         NaN   16.693565  ...        NaN   
2000-01-03   18.250000    9.306484         NaN   16.274673  ...        NaN   
2000-01-04   17.812500    9.040581         NaN   14.909401  ...        NaN   
2000-01-05   18.000000    9.023960         NaN   15.204174  ...        NaN   
2000-01-06   18.031250    9.339719         NaN   15.328289  ...        NaN   
...                ...         ...         ...         ...  ...        ...   
2022-08-24  268.040009  105.440002  306.260010  405.649994  ...  1337400.0   
2022-08-25  274.450012  105.889999  309.769989  403.929993  ...  1486700.0   
2022-08-26  258.119995  101.900002  298.130005  381.019989  ...  1599000.0   
2022-08-29  256.980011  101.839996  295.140015  375.260010  ...  1127600.0   
2022-08-30  258.649994  102.199997  292.500000  375.070007  ...  1887700.0   

                                                                             \
Ticker            XEL         XOM       XRAY       XYL        YUM       ZBH   
Date                                                                          
1999-12-31   262900.0   2758600.0   241200.0       NaN  1395451.0       NaN   
2000-01-03  2738600.0  13458200.0   582300.0       NaN  3033493.0       NaN   
2000-01-04   425200.0  14510800.0   317700.0       NaN  3315031.0       NaN   
2000-01-05   500200.0  17485000.0  1188000.0       NaN  4642602.0       NaN   
2000-01-06   344100.0  19461600.0   534300.0       NaN  3947658.0       NaN   
...               ...         ...        ...       ...        ...       ...   
2022-08-24  1431100.0  17050300.0  4222500.0  412300.0  1300100.0  620600.0   
2022-08-25  1553600.0  15305700.0  1674300.0  487000.0  1230600.0  530700.0   
2022-08-26  2413300.0  15531400.0  2240200.0  628700.0  1542600.0  627700.0   
2022-08-29  2278200.0  23059200.0  1555700.0  873200.0  1472900.0  556700.0   
2022-08-30  2538400.0  17086800.0  2640000.0  643700.0  1230700.0  782900.0   

                                             
Ticker           ZBRA       ZION        ZTS  
Date                                         
1999-12-31   278100.0   158100.0        NaN  
2000-01-03  1055700.0  1199600.0        NaN  
2000-01-04   522450.0   816100.0        NaN  
2000-01-05   612225.0  1124700.0        NaN  
2000-01-06   263925.0  1112100.0        NaN  
...               ...        ...        ...  
2022-08-24   210900.0   549300.0  1206400.0  
2022-08-25   179300.0   864600.0  1088800.0  
2022-08-26   284300.0  1215200.0  1468000.0  
2022-08-29   235300.0  1221200.0  1236900.0  
2022-08-30   311700.0   941600.0  1918800.0  

[5703 rows x 3006 columns]

In [9]:
o = data['Open'].unstack(level = 0)
h = data['High'].unstack(level = 0)
l = data['Low'].unstack(level = 0)
c = data['Adj Close'].unstack(level = 0)
v = data['Volume'].unstack(level = 0)

# Alpha031

# 1.1:

$-1 * rank(delta(close,10))$

這一段主要目標為今日收盤價跟10天前(也就是2週前)收盤價之漲跌多寡，接著透過排名並且乘上負號後，絕對值大的會是相對漲較多或是跌較少的股票，值大的會是漲較少或跌較多的股票。 <br>

In [10]:
close_diff = c.diff(10) #今天減掉10天前價格

In [11]:
close_diff

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,1.089996,-1.08,-14.649628,-1.710007,-3.029999,3.101807,-25.299988,-4.829994,-10.220001,-32.750000,...,-2.529999,0.298050,8.010002,-2.680000,-4.335716,-4.487999,-5.270004,-9.529999,0.281929,-11.429993
2022-08-25,4.059998,-0.57,-16.285049,1.539993,-2.750000,2.660004,-16.479980,-4.599998,-4.650024,-30.960022,...,-2.620003,1.460442,5.899994,-2.410000,-3.578209,-0.885658,-3.059998,-5.859985,0.457909,-7.649994
2022-08-26,-3.680008,-1.51,-28.684708,-8.480011,-6.250000,-4.309998,-35.470001,-9.580002,-22.309998,-64.650024,...,-6.409996,-1.023300,3.870003,-4.040001,-10.375809,-5.423462,-9.709999,-38.579987,-1.375355,-15.779999


以下進行排序

In [12]:
close_10_rank = close_diff.rank(axis=1, pct=True)

In [13]:
close_10_rank

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.772455,0.512974,0.095808,0.437126,0.339321,0.854291,0.043912,0.261477,0.139721,0.023952,...,0.369261,0.700599,0.950100,0.361277,0.283433,0.277445,0.245509,0.145709,0.696607,0.117764
2022-08-25,0.854291,0.469062,0.045908,0.734531,0.237525,0.814371,0.039920,0.177645,0.175649,0.007984,...,0.241517,0.726547,0.896208,0.253493,0.191617,0.417166,0.215569,0.137725,0.624750,0.107784
2022-08-26,0.576846,0.756487,0.085828,0.331337,0.401198,0.522954,0.053892,0.297405,0.126747,0.013972,...,0.389222,0.796407,0.952096,0.546906,0.265469,0.453094,0.287425,0.037924,0.774451,0.173653


以下將排名乘負號

In [14]:
reverse_close_10_rank = -close_10_rank

In [15]:
reverse_close_10_rank

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,-0.772455,-0.512974,-0.095808,-0.437126,-0.339321,-0.854291,-0.043912,-0.261477,-0.139721,-0.023952,...,-0.369261,-0.700599,-0.950100,-0.361277,-0.283433,-0.277445,-0.245509,-0.145709,-0.696607,-0.117764
2022-08-25,-0.854291,-0.469062,-0.045908,-0.734531,-0.237525,-0.814371,-0.039920,-0.177645,-0.175649,-0.007984,...,-0.241517,-0.726547,-0.896208,-0.253493,-0.191617,-0.417166,-0.215569,-0.137725,-0.624750,-0.107784
2022-08-26,-0.576846,-0.756487,-0.085828,-0.331337,-0.401198,-0.522954,-0.053892,-0.297405,-0.126747,-0.013972,...,-0.389222,-0.796407,-0.952096,-0.546906,-0.265469,-0.453094,-0.287425,-0.037924,-0.774451,-0.173653


# 1.2

$\text{decay_linear}(1.1Part,10)$

這邊將個別股票在10天當中的排名情況，以加權平均(並且日期越早，權重遞減)方式做加總。<br>
以下使用talib.WMA作為decay_linear的實作。

In [17]:
reverse_close_10_rank_WMA = reverse_close_10_rank.apply(lambda x: talib.WMA(x, timeperiod=10))

In [18]:
reverse_close_10_rank_WMA

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,-0.632735,-0.307313,-0.731918,-0.714643,-0.371584,-0.825113,-0.060969,-0.291091,-0.538160,-0.364108,...,-0.247069,-0.567338,-0.691671,-0.245781,-0.457304,-0.167120,-0.278752,-0.324333,-0.666340,-0.075268
2022-08-25,-0.694320,-0.340428,-0.600327,-0.710724,-0.357431,-0.833787,-0.049936,-0.267719,-0.449501,-0.262058,...,-0.232081,-0.602722,-0.757612,-0.247033,-0.388387,-0.219706,-0.252404,-0.290328,-0.663945,-0.084123
2022-08-26,-0.695772,-0.423880,-0.488332,-0.635638,-0.369516,-0.784177,-0.043150,-0.266957,-0.363890,-0.178625,...,-0.250626,-0.646779,-0.824714,-0.302776,-0.344765,-0.272473,-0.246634,-0.236327,-0.688332,-0.103466


# 1.3

$\text{rank}(1.2Part)$

將做完10天加權平均後的排名值做排序，因為排名值有取負號，故加權平均後值較大的話，為10天當中近期漲得較少，跌的較多的股票

In [21]:
reverse_close_10_rank_WMA_rank = reverse_close_10_rank_WMA.rank(axis=1, pct=True)

In [22]:
reverse_close_10_rank_WMA_rank

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.282520,0.794715,0.172764,0.191057,0.709350,0.089431,0.989837,0.829268,0.402439,0.719512,...,0.882114,0.378049,0.223577,0.886179,0.560976,0.949187,0.845528,0.762195,0.252033,0.987805
2022-08-25,0.213415,0.737805,0.335366,0.199187,0.709350,0.097561,0.991870,0.857724,0.560976,0.863821,...,0.894309,0.333333,0.166667,0.882114,0.662602,0.910569,0.876016,0.835366,0.247967,0.979675
2022-08-26,0.209350,0.621951,0.532520,0.298780,0.711382,0.123984,0.991870,0.851626,0.715447,0.918699,...,0.865854,0.288618,0.083333,0.802846,0.741870,0.843496,0.876016,0.884146,0.221545,0.963415


# 2

$-1 * rank(delta(close,3))$:

將今天收盤價減掉3天前收盤價，並且做排名之後取負號。<br>
值大的話表示，今天跟三天前比起來漲的少，跌得多。

In [27]:
rev_close_3_rank = c.diff(3).rank(axis=1, pct=True).mul(-1)

In [28]:
rev_close_3_rank

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,-0.024658,NaN,NaN,-0.789041,NaN,-0.838356,-0.495890,-0.375342,NaN,-0.191781,...,NaN,-0.906849,-0.849315,-0.835616,NaN,-0.558904,NaN,-0.306849,-0.043836,NaN
2000-01-06,-0.024658,NaN,NaN,-0.435616,NaN,-0.753425,-0.389041,-0.616438,NaN,-0.183562,...,NaN,-0.813699,-0.969863,-0.682192,NaN,-0.452055,NaN,-0.156164,-0.142466,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,-0.305389,-0.846307,-0.009980,-0.303393,-0.307385,-0.327345,-0.441118,-0.249501,-0.097804,-0.017964,...,-0.732535,-0.612774,-0.970060,-0.632735,-0.369261,-0.518962,-0.351297,-0.095808,-0.644711,-0.153693
2022-08-25,-0.628743,-0.383234,-0.005988,-0.660679,-0.086826,-0.520958,-0.956088,-0.055888,-0.163673,-0.013972,...,-0.810379,-0.185629,-0.836327,-0.265469,-0.355289,-0.626747,-0.361277,-0.960080,-0.493014,-0.341317
2022-08-26,-0.297405,-0.796407,-0.011976,-0.231537,-0.303393,-0.396208,-0.199601,-0.201597,-0.065868,-0.007984,...,-0.914172,-0.678643,-0.774451,-0.624750,-0.267465,-0.371257,-0.403194,-0.061876,-0.855289,-0.307385


# 3

$rank(Part 1 + Part 2)$

注意Part 1 的值為正，Part 2 的值為負，會將Part1跟Part2的力量抵消。<br>
若Part1的力量較強，則正負相消後值為正，代表10天當中近期漲得較少，跌的較多。3天以來漲的較少，跌得較多的股票。<br>

In [29]:
price_rank = (reverse_close_10_rank_WMA_rank + rev_close_3_rank).rank(axis=1,pct=True)

In [30]:
price_rank

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.526423,0.491870,0.680894,0.428862,0.796748,0.298780,0.857724,0.882114,0.758130,0.932927,...,0.668699,0.302846,0.050813,0.737805,0.697154,0.810976,0.831301,0.914634,0.168699,0.965447
2022-08-25,0.191057,0.798780,0.782520,0.166667,0.934959,0.186992,0.471545,0.977642,0.853659,0.987805,...,0.550813,0.626016,0.079268,0.930894,0.774390,0.747967,0.898374,0.333333,0.274390,0.936992
2022-08-26,0.445122,0.380081,0.839431,0.589431,0.786585,0.294715,0.943089,0.884146,0.880081,0.981707,...,0.475610,0.203252,0.077236,0.674797,0.823171,0.817073,0.821138,0.955285,0.093496,0.888211


# 4.1

$correlation(adv20, low, 12)$

adv20為20天之daily dollar volume平均<br>
dollar volume部分使用開盤收盤最高最低價之平均 * 日交易量<br>
corr為判斷12天內，一個月的daily volume平均跟最低價之相關性。<br>
相關性高可以預期走勢會相同，亦即量下降則最低價下跌。

In [31]:
vwap = o.add(h).add(l).add(c).div(4)

In [32]:
daily_dollar_volume = vwap.mul(v)

In [33]:
daily_dollar_volume

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,1.042795e+08,NaN,NaN,1.430047e+08,NaN,9.519963e+05,2.116969e+06,4.596122e+07,NaN,4.797391e+07,...,NaN,4.344436e+06,9.704321e+07,1.852335e+06,NaN,8.964742e+06,NaN,7.029750e+06,8.603901e+06,NaN
2000-01-03,2.397149e+08,NaN,NaN,4.958970e+08,NaN,1.018703e+07,3.418375e+06,1.512575e+08,NaN,1.218881e+08,...,NaN,4.486643e+07,4.654759e+08,4.448383e+06,NaN,1.921471e+07,NaN,2.685070e+07,6.293667e+07,NaN
2000-01-04,2.204856e+08,NaN,NaN,4.661496e+08,NaN,3.231462e+06,6.175875e+05,1.482738e+08,NaN,1.212939e+08,...,NaN,6.917583e+06,4.895454e+08,2.412124e+06,NaN,2.036857e+07,NaN,1.278188e+07,4.050655e+07,NaN
2000-01-05,2.538714e+08,NaN,NaN,7.055467e+08,NaN,2.906810e+06,2.154756e+06,1.598948e+08,NaN,2.239702e+08,...,NaN,8.361587e+06,6.064134e+08,9.104398e+06,NaN,2.850151e+07,NaN,1.492086e+07,5.438813e+07,NaN
2000-01-06,1.056815e+08,NaN,NaN,6.672432e+08,NaN,1.034839e+07,1.501786e+06,2.429862e+08,NaN,1.571514e+08,...,NaN,5.831934e+06,7.020292e+08,4.098675e+06,NaN,2.409559e+07,NaN,6.409345e+06,5.475666e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,1.362061e+08,2.479917e+08,6.548415e+08,9.007818e+09,6.952682e+08,6.742423e+07,7.227841e+07,4.520239e+08,5.191536e+08,7.699970e+08,...,8.318962e+07,1.075865e+08,1.674851e+09,1.448212e+08,3.962557e+07,1.489526e+08,6.778503e+07,6.723861e+07,3.080337e+07,1.965799e+08
2022-08-25,1.311628e+08,3.166670e+08,1.808002e+08,8.672522e+09,4.583139e+08,1.115065e+08,5.563013e+07,5.533182e+08,5.312821e+08,1.232088e+09,...,9.584383e+07,1.174543e+08,1.515149e+09,5.790567e+07,4.718539e+07,1.423558e+08,5.893954e+07,5.847018e+07,4.918709e+07,1.788572e+08
2022-08-26,1.665151e+08,3.268027e+08,1.535635e+08,1.320228e+10,5.291210e+08,1.137879e+08,7.735385e+07,5.655200e+08,5.172311e+08,1.413984e+09,...,1.023360e+08,1.821758e+08,1.529726e+09,7.705728e+07,5.996771e+07,1.764966e+08,6.898894e+07,9.090350e+07,6.933020e+07,2.391005e+08


In [34]:
adv20 = daily_dollar_volume.rolling(20).mean()

In [35]:
adv20

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,1.983603e+08,3.775554e+08,1.396650e+08,1.104309e+10,7.992972e+08,1.465535e+08,6.803440e+07,3.964449e+08,4.884142e+08,9.116792e+08,...,1.367398e+08,1.618425e+08,1.784342e+09,7.134411e+07,8.413999e+07,2.098272e+08,1.122285e+08,1.165141e+08,5.923839e+07,2.532042e+08
2022-08-25,1.928687e+08,3.688511e+08,1.414530e+08,1.083976e+10,7.729025e+08,1.475008e+08,6.836873e+07,3.980416e+08,4.872989e+08,9.250776e+08,...,1.364819e+08,1.584161e+08,1.766427e+09,7.130314e+07,8.166802e+07,2.044893e+08,1.026346e+08,1.139805e+08,5.780108e+07,2.502624e+08
2022-08-26,1.864994e+08,3.672476e+08,1.441527e+08,1.067711e+10,6.719020e+08,1.482284e+08,6.863883e+07,3.994201e+08,4.846391e+08,9.414771e+08,...,1.359695e+08,1.579163e+08,1.700470e+09,7.168320e+07,8.105434e+07,2.026213e+08,9.738122e+07,1.104619e+08,5.661224e+07,2.462845e+08


In [36]:
adv20_l_corr = adv20.rolling(12).corr(l)

In [37]:
adv20_l_corr

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.629703,0.747953,-0.899427,-0.565104,0.074313,0.688479,0.410119,0.432841,-0.163715,0.791182,...,0.715801,-0.568499,0.161847,-0.934266,0.746438,-0.317196,0.742416,0.627982,0.479357,0.534656
2022-08-25,0.575039,0.840655,-0.945228,-0.673570,0.316051,0.690184,0.390059,0.624414,0.105347,0.820640,...,0.804343,-0.372175,0.328272,-0.931102,0.774434,-0.285662,0.824433,0.722029,0.597760,0.707057
2022-08-26,0.462259,0.859936,-0.970655,-0.206826,0.724374,0.524033,0.243335,0.640043,0.256437,0.692420,...,0.811411,-0.146069,0.087139,-0.938649,0.850040,0.223706,0.891513,0.844541,0.705704,0.778228


# 4.2

$sign(scale(correlation(..)))$

將個別股票的相關性的值做標準化後取出正負號。<br>
其實直接取sign也可以，不用scale。<br>
取出結果為正相關或負相關。

In [38]:
scale_corr = adv20_l_corr.div(adv20_l_corr.abs().sum(axis=1), axis=0)

In [39]:
scale_corr

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.002625,0.003118,-0.003750,-0.002356,0.000310,0.002870,0.001710,0.001804,-0.000683,0.003298,...,0.002984,-0.002370,0.000675,-0.003895,0.003112,-0.001322,0.003095,0.002618,0.001998,0.002229
2022-08-25,0.002219,0.003243,-0.003647,-0.002599,0.001219,0.002663,0.001505,0.002409,0.000406,0.003166,...,0.003103,-0.001436,0.001267,-0.003592,0.002988,-0.001102,0.003181,0.002786,0.002306,0.002728
2022-08-26,0.001643,0.003056,-0.003449,-0.000735,0.002574,0.001862,0.000865,0.002274,0.000911,0.002460,...,0.002883,-0.000519,0.000310,-0.003335,0.003021,0.000795,0.003168,0.003001,0.002508,0.002765


In [40]:
sign_scale_corr = np.sign(scale_corr)

In [41]:
sign_scale_corr

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,...,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
2022-08-25,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
2022-08-26,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0


# 5.1

$Part3 + Part4$

Part3的結果為正負相消後的值，若為正，代表10天當中近期漲得較少，跌的較多。3天以來漲的較少，跌得較多的股票。<br>
Part4的結果為12天內，一個月的daily volume平均跟最低價之正相關或負相關，也就是價量是正相關或負相關。<br>
將兩者加總後，值大為近期漲得較少，跌的較多，交易量與最低價正相關之股票。

In [42]:
price_rank_volume_corr = price_rank.add(sign_scale_corr)

In [43]:
price_rank_volume_corr

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,1.526423,1.491870,-0.319106,-0.571138,1.796748,1.298780,1.857724,1.882114,-0.241870,1.932927,...,1.668699,-0.697154,1.050813,-0.262195,1.697154,-0.189024,1.831301,1.914634,1.168699,1.965447
2022-08-25,1.191057,1.798780,-0.217480,-0.833333,1.934959,1.186992,1.471545,1.977642,1.853659,1.987805,...,1.550813,-0.373984,1.079268,-0.069106,1.774390,-0.252033,1.898374,1.333333,1.274390,1.936992
2022-08-26,1.445122,1.380081,-0.160569,-0.410569,1.786585,1.294715,1.943089,1.884146,1.880081,1.981707,...,1.475610,-0.796748,1.077236,-0.325203,1.823171,1.817073,1.821138,1.955285,1.093496,1.888211


# 5.2

$rank(Part5.1)$

將5.1之結果做排序，值大為近期漲得較少，跌的較多，交易量與最低價正相關之股票

In [44]:
alpha_031 = price_rank_volume_corr.rank(axis=1,pct=True)

In [45]:
alpha_031

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-24,0.686992,0.660569,0.296748,0.195122,0.855691,0.565041,0.894309,0.908537,0.325203,0.943089,...,0.774390,0.134146,0.432927,0.317073,0.794715,0.341463,0.880081,0.930894,0.500000,0.971545
2022-08-25,0.473577,0.861789,0.292683,0.063008,0.963415,0.471545,0.648374,0.985772,0.906504,0.991870,...,0.705285,0.231707,0.406504,0.327236,0.841463,0.278455,0.936992,0.560976,0.528455,0.965447
2022-08-26,0.585366,0.540650,0.254065,0.184959,0.835366,0.475610,0.959350,0.916667,0.912602,0.987805,...,0.607724,0.065041,0.339431,0.209350,0.863821,0.857724,0.861789,0.969512,0.351626,0.918699


# Intuition of This Alpha Factor

初步想法： 找出近期漲得較少，跌的較多，交易量與最低價正相關之股票。<br>
透過後續日期判斷交易量的增減，預期最低價會上升或下跌。<br>
情況一：交易量下跌，預期最低價下跌，做空。<br>
情況二：交易量上升，預期最低價上升，做多。<br>
問題在於：情況一二之出場時機。 <br>
出場時機的部分於Strategy&Analysis的檔案說明。